In [ ]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
import os

# Directory containing the shapefiles
shapefile_dir = '/path/to/shapefile/directory/'

# output directory for tif files
output_raster_dir = '/path/to/raster/output/'

# Iterate through each shapefile in the directory
for shapefile in os.listdir(shapefile_dir):
    if shapefile.endswith('.shp'):
        shp_path = os.path.join(shapefile_dir, shapefile)
        gdf = gpd.read_file(shp_path)
        
        
        gdf['landclass'] = gdf['landclass'].astype(int)
        gdf = gdf.dropna(subset=['landclass'])
        
        raster_file = os.path.join(output_raster_dir, f"{os.path.splitext(shapefile)[0]}_raster.tif")
        
        pixel_size = 2
        minx, miny, maxx, maxy = gdf.total_bounds
        transform = from_origin(minx, maxy, pixel_size, pixel_size)

        width = int((maxx - minx) / pixel_size)
        height = int((maxy - miny) / pixel_size)

        # Write the raster data
        with rasterio.open(raster_file, 'w', driver='GTiff', 
                           height=height, width=width, count=1, dtype='uint8',
                           crs=gdf.crs, transform=transform) as dst:

            # Rasterizing
            burned = rasterize(
                ((geom, value) for geom, value in zip(gdf.geometry, gdf['landclass'])),
                out_shape=(height, width),
                transform=transform,
                fill=0,  
                dtype='uint8'
            )
            
            dst.write(burned, 1) 
